In [ ]:
import tenseal as ts
import yapf.yapflib.yapf_api as yapf
from google.cloud import storage
from seal import EncryptionParameters, SEALContext, KeyGenerator, PublicKey, Encryptor, Ciphertext, Evaluator, Decryptor
from Pyfhel import Pyfhel, PyPtxt, PyCtxt
from deepface import DeepFace
import base64
import deepface.commons.distance as dst

In [ ]:
client = storage.Client()

In [ ]:
HE = Pyfhel()
HE.contextGen(p=65537, m=1024, flagBatching=True)  
HE.keyGen() 

In [ ]:
parms = EncryptionParameters(scheme_type.BFV)
parms.set_poly_modulus_degree(4096)
parms.set_coeff_modulus(CoeffModulus.BFVDefault(4096))
parms.set_plain_modulus(256)
context = SEALContext.Create(parms)
keygen = KeyGenerator(context)
public_key = keygen.public_key()
encryptor = Encryptor(context, public_key)

In [ ]:
def write_data(bucket_name, file_name, data):
    if type(data) == bytes:
        data = base64.b64encode(data)
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(file_name)
    blob.upload_from_string(data)

In [ ]:
def read_data(bucket_name, file_name):
    bucket = client.get_bucket(bucket_name)
    blob = bucket.get_blob(file_name)
    data = blob.download_as_text()
    return base64.b64decode(data)


In [ ]:
def main():
    img1_path = "../pics/andrew1.png"
    img2_path = "../pics/andrew4.png"
    img1_embedding = DeepFace.represent(img1_path, model_name='Facenet')[0]["embedding"]
    img2_embedding = DeepFace.represent(img2_path, model_name='Facenet')[0]["embedding"]

    img1_embedding_encoded = yapf.FormatCode(img1_embedding)
    img2_embedding_encoded = yapf.FormatCode(img2_embedding)

    img1_embedding_encrypted = Ciphertext()
    encryptor.encrypt(encoder.encode(img1_embedding_encoded), img1_embedding_encrypted)
    img2_embedding_encrypted = Ciphertext()
    encryptor.encrypt(encoder.encode(img2_embedding_encoded), img2_embedding_encrypted)

    smc_result = HE.add(img1_embedding_encrypted, img2_embedding_encrypted)

    img1_embedding_decrypted = HE.decrypt(img1_embedding_encrypted)
    img2_embedding_decrypted = HE.decrypt(img2_embedding_encrypted)
    smc_result_decrypted = HE.decrypt(smc_result)

    distance = dst.findEuclideanDistance(img1_embedding_decrypted, img2_embedding_decrypted)
    squared_distance = distance * distance
    if smc_result_decrypted < 100:
        print("They are the same person")
    else:
        print("They are different persons")

    print("Euclidean squared - traditional: ", squared_distance)
    print("Euclidean squared - homomorphic: ", smc_result_decrypted)
    assert abs(squared_distance - smc_result_decrypted) < 0.00001

    write_data("iotHomomorphicBucket", "enc_v1.txt", img1_embedding_encrypted)
    write_data("iotHomomorphicBucket", "enc_v2.txt", img2_embedding_encrypted)
    write_data("iotHomomorphicBucket", "result.txt", smc_result)

    img1_embedding_encrypted = read_data("iotHomomorphicBucket", "enc_v1.txt")
    img2_embedding_encrypted = read_data("iotHomomorphicBucket", "enc_v2.txt")
    smc_result = read_data("iotHomomorphicBucket", "result.txt")

if __name__ == "__main__":
    main()